# Furniture E-commerce Customer Churn Prediction - Data Preprocessing
# Project: furniture-ecommerce-churn-prediction-dashboard
# Author: Hansel Liebrata
# File: notebooks/02_data_preprocessing.ipynb

## Overview
#This notebook focuses on preprocessing the e-commerce user churn data based on insights from our exploratory analysis. We will:

1. Handle Missing Values and Outliers
2. Feature Scaling and Transformation
3. Create Behavioral Features
4. Engineer Temporal Features
5. Develop Value-Based Features
6. Generate Final Feature Set

The preprocessed data will be used for model development in subsequent notebooks.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("E-COMMERCE CUSTOMER CHURN PREDICTION")
print("=" * 60)
print("Phase 2: Data Preprocessing")
print("=" * 60)

E-COMMERCE CUSTOMER CHURN PREDICTION
Phase 2: Data Preprocessing


In [20]:
def load_data():
    """Load the e-commerce user churn dataset"""
    print("\nDATA LOADING")
    print("-" * 40)
    
    file_path = '../data/raw/ecom-user-churn-data.csv'
    df = pd.read_csv(file_path)
    print(f"Dataset loaded successfully: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    return df

def handle_missing_values(df):
    """Handle missing values in the dataset"""
    print("\nHANDLING MISSING VALUES")
    print("-" * 40)
    
    # Check for missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print("\nMissing values found:")
        print(missing[missing > 0])
        
        # Handle missing values based on feature type
        for column in df.columns:
            if df[column].isnull().sum() > 0:
                if column in ['ses_rec_sd', 'ses_rec_cv']:
                    # For variation metrics, missing values likely mean no variation (single session)
                    df[column].fillna(0, inplace=True)
                elif 'avg' in column or 'mean' in column:
                    # For averages, use median
                    df[column].fillna(df[column].median(), inplace=True)
                else:
                    # For other metrics, use 0 as it likely means no activity
                    df[column].fillna(0, inplace=True)
    else:
        print("No missing values found in the dataset")
    
    return df

def handle_outliers(df):
    """Handle outliers using capping"""
    print("\nHANDLING OUTLIERS")
    print("-" * 40)
    
    # List of numerical columns to check for outliers
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    # Exclude certain columns from outlier treatment
    exclude_cols = ['visitorid', 'target_class']
    cols_to_treat = [col for col in numerical_cols if col not in exclude_cols]
    
    for column in cols_to_treat:
        # Calculate quartiles and IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers
        outliers = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()
        
        if outliers > 0:
            print(f"\nOutliers found in {column}: {outliers}")
            
            # Cap the outliers
            df[column] = df[column].clip(lower_bound, upper_bound)
    
    return df

# Load and preprocess the data
df = load_data()
df = handle_missing_values(df)
#df = handle_outliers(df)

# Display the shape of processed dataset
print("\nProcessed dataset shape:", df.shape)


DATA LOADING
----------------------------------------
Dataset loaded successfully: 49,358 rows × 49 columns

HANDLING MISSING VALUES
----------------------------------------
No missing values found in the dataset

Processed dataset shape: (49358, 49)


In [21]:
df.describe()

,visitorid,ses_rec,ses_rec_avg,ses_rec_sd,ses_rec_cv,user_rec,ses_n,ses_n_r,int_n,int_n_r,...,int_cat16_n,int_cat17_n,int_cat18_n,int_cat19_n,int_cat20_n,int_cat21_n,int_cat22_n,int_cat23_n,int_cat24_n,target_class
count,4.935800e+04,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,...,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000,49358.000000
mean,7.072986e+05,15.454840,11.231611,2.711961,-0.020100,33.822947,3.366445,0.172372,6.716277,1.720975,...,0.955792,0.773714,0.382977,0.732424,0.503343,0.447020,2.102577,0.038130,0.099579,0.885591
std,4.072098e+05,9.184645,18.162743,6.583917,0.917701,25.237703,7.380573,0.372614,38.528882,1.455885,...,6.086722,5.003517,4.569604,4.977989,3.259194,3.873684,16.273213,0.593681,1.135149,0.318311
min,3.700000e+01,0.000000,0.000000,0.000000,-1.000000,0.000000,2.000000,-1.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.532920e+05,7.000000,0.000000,0.000000,-1.000000,16.000000,2.000000,0.060606,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,7.100910e+05,16.000000,2.250000,0.000000,0.000000,26.000000,2.000000,0.090909,3.000000,1.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.060355e+06,23.000000,14.250000,1.000000,0.638646,46.000000,3.000000,0.166667,6.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000
max,1.407573e+06,31.000000,99.000000,47.500000,11.525121,99.000000,475.000000,18.000000,5549.000000,59.000000,...,576.000000,445.000000,481.000000,564.000000,317.000000,420.000000,2282.000000,54.000000,105.000000,1.000000


In [22]:
def engineer_behavioral_features(df):
    """Create behavioral features based on user activity patterns"""
    print("\nENGINEERING BEHAVIORAL FEATURES")
    print("-" * 40)
    
    # 1. Engagement Features
    df['engagement_ratio'] = (df['int_n'] / df['ses_n']).clip(0, None)
    df['conversion_rate'] = (df['tran_n'] / df['int_n']).fillna(0)
    df['avg_items_per_interaction'] = df['int_itm_n_avg']
    
    # 2. Session Patterns
    df['session_consistency'] = 1 - df['ses_rec_cv'].clip(0, 1)
    df['weekend_preference'] = (df['ses_wknd_r'] > 0.5).astype(int)
    
    # 3. Category Engagement
    category_cols = [col for col in df.columns if col.startswith('int_cat') and col.endswith('_n')]
    df['category_diversity'] = (df[category_cols] > 0).sum(axis=1) / len(category_cols)
    
    # 4. Value Features
    df['avg_transaction_value'] = (df['rev_sum'] / df['tran_n']).fillna(0)
    
    # Create value segments handling zero values
    zero_mask = df['rev_sum'] == 0
    non_zero_values = df.loc[~zero_mask, 'rev_sum']
    
    if len(non_zero_values) > 0:
        # Create segments for non-zero values
        try:
            # Try to create 3 segments for non-zero values
            labels = ['Low', 'Medium', 'High']
            non_zero_segments = pd.qcut(
                non_zero_values,
                q=3,
                labels=labels,
                duplicates='drop'
            )
            # Assign segments
            df['value_segment'] = 'No Purchase'
            df.loc[~zero_mask, 'value_segment'] = non_zero_segments
        except ValueError:
            # If we can't create 3 segments, use binary segmentation
            df['value_segment'] = np.where(zero_mask, 'No Purchase', 'Has Purchase')
    else:
        # If all values are zero
        df['value_segment'] = 'No Purchase'
    
    print("Created new behavioral features:")
    print("- engagement_ratio")
    print("- conversion_rate")
    print("- avg_items_per_interaction")
    print("- session_consistency")
    print("- weekend_preference")
    print("- category_diversity")
    print("- avg_transaction_value")
    print("- value_segment")
    
    return df

def engineer_temporal_features(df):
    """Create temporal features based on user activity timing"""
    print("\nENGINEERING TEMPORAL FEATURES")
    print("-" * 40)
    
    # 1. Recency Features
    df['recency_score'] = 1 / (1 + df['ses_rec'])  # Higher score for more recent activity
    df['user_lifetime'] = df['user_rec']
    
    # 2. Activity Patterns
    df['activity_regularity'] = 1 / (1 + df['ses_rec_cv'])
    df['peak_hour_activity'] = (df['ses_ho_avg'] >= 9) & (df['ses_ho_avg'] <= 18)
    
    # 3. Time-based Segments
    # Handle potential duplicate edges in recency segmentation
    try:
        df['recency_segment'] = pd.qcut(
            df['ses_rec'],
            q=4,
            labels=['Very Recent', 'Recent', 'Moderate', 'Old'],
            duplicates='drop'
        )
    except ValueError:
        # If we can't create 4 segments, try with fewer segments
        unique_values = df['ses_rec'].nunique()
        if unique_values > 2:
            df['recency_segment'] = pd.qcut(
                df['ses_rec'],
                q=3,
                labels=['Recent', 'Moderate', 'Old'],
                duplicates='drop'
            )
        else:
            df['recency_segment'] = pd.cut(
                df['ses_rec'],
                bins=2,
                labels=['Recent', 'Old']
            )
    
    print("Created new temporal features:")
    print("- recency_score")
    print("- user_lifetime")
    print("- activity_regularity")
    print("- peak_hour_activity")
    print("- recency_segment")
    
    return df

def create_risk_features(df):
    """Create risk-related features based on user behavior"""
    print("\nCREATING RISK FEATURES")
    print("-" * 40)
    
    # 1. Basic Risk Indicators
    df['activity_decline'] = df['ses_rec'] > df['ses_rec_avg']
    df['low_engagement'] = df['engagement_ratio'] < df['engagement_ratio'].median()
    
    # 2. Risk Score Components
    # Handle division by zero
    max_rec = df['ses_rec'].max() if df['ses_rec'].max() > 0 else 1
    max_eng = df['engagement_ratio'].max() if df['engagement_ratio'].max() > 0 else 1
    max_rev = df['rev_sum'].max() if df['rev_sum'].max() > 0 else 1
    
    df['recency_risk'] = (df['ses_rec'] / max_rec).clip(0, 1)
    df['engagement_risk'] = (1 - df['engagement_ratio'] / max_eng).clip(0, 1)
    df['value_risk'] = (1 - df['rev_sum'] / max_rev).clip(0, 1)
    
    # 3. Composite Risk Score
    df['churn_risk_score'] = (
        df['recency_risk'] * 0.4 +
        df['engagement_risk'] * 0.3 +
        df['value_risk'] * 0.3
    )
    
    # 4. Risk Segments
    try:
        df['risk_segment'] = pd.qcut(
            df['churn_risk_score'],
            q=5,
            labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
            duplicates='drop'
        )
    except ValueError:
        # If we can't create 5 segments, try with 3 segments
        df['risk_segment'] = pd.qcut(
            df['churn_risk_score'],
            q=3,
            labels=['Low', 'Medium', 'High'],
            duplicates='drop'
        )
    
    print("Created new risk features:")
    print("- activity_decline")
    print("- low_engagement")
    print("- recency_risk")
    print("- engagement_risk")
    print("- value_risk")
    print("- churn_risk_score")
    print("- risk_segment")
    
    return df

def scale_numerical_features(df):
    """Scale numerical features using RobustScaler"""
    print("\nSCALING NUMERICAL FEATURES")
    print("-" * 40)
    
    # Identify numerical columns to scale
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cols_to_scale = [col for col in numerical_cols if col not in ['visitorid', 'target_class']]
    
    # Handle infinite values and extremely large numbers
    for col in cols_to_scale:
        # Replace infinite values with NaN
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        
        # For any remaining NaN values, replace with the median
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())
        
        # Clip extremely large values to a reasonable range
        # Using 99th percentile as the upper bound
        upper_bound = df[col].quantile(0.99)
        df[col] = df[col].clip(None, upper_bound)
    
    # Initialize scaler
    scaler = RobustScaler()
    
    # Scale features
    df_scaled = df.copy()
    df_scaled[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    # Save scaler for later use
    with open('../models/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    
    print(f"Scaled {len(cols_to_scale)} numerical features")
    print("Scaler saved to ../models/scaler.pkl")
    
    return df_scaled

In [30]:
# Apply feature engineering
print("\nAPPLYING FEATURE ENGINEERING")
print("=" * 60)

# Engineer features
df = engineer_behavioral_features(df)
df = engineer_temporal_features(df)
df = create_risk_features(df)

# Scale features
df_scaled = scale_numerical_features(df)

print("\nFeature engineering complete!")
print(f"Final dataset shape: {df_scaled.shape}")

# Display sample of processed data
print("\nSample of processed data:")
print(df_scaled.head())


APPLYING FEATURE ENGINEERING

ENGINEERING BEHAVIORAL FEATURES
----------------------------------------
Created new behavioral features:
- engagement_ratio
- conversion_rate
- avg_items_per_interaction
- session_consistency
- weekend_preference
- category_diversity
- avg_transaction_value
- value_segment

ENGINEERING TEMPORAL FEATURES
----------------------------------------
Created new temporal features:
- recency_score
- user_lifetime
- activity_regularity
- peak_hour_activity
- recency_segment

CREATING RISK FEATURES
----------------------------------------
Created new risk features:
- activity_decline
- low_engagement
- recency_risk
- engagement_risk
- value_risk
- churn_risk_score
- risk_segment

SCALING NUMERICAL FEATURES
----------------------------------------
Scaled 61 numerical features
Scaler saved to ../models/scaler.pkl

Feature engineering complete!
Final dataset shape: (49358, 69)

Sample of processed data:
   visitorid  ses_rec  ses_rec_avg  ses_rec_sd  ses_rec_cv  user

# Data Preprocessing Summary

## Steps Completed:

1. Data Cleaning
   - Handled missing values using appropriate strategies
   - Treated outliers using IQR-based capping
   - Removed invalid entries (if any)

2. Feature Engineering
   - Created behavioral features:
     * Engagement metrics
     * Session patterns
     * Category diversity
     * Value-based features
   
   - Added temporal features:
     * Recency scores
     * Activity patterns
     * Time-based segments
   
   - Developed risk features:
     * Risk indicators
     * Composite risk score
     * Risk segmentation

3. Feature Scaling
   - Applied RobustScaler to numerical features
   - Preserved categorical features
   - Saved scaler for future use

4. Data Storage
   - Saved preprocessed dataset
   - Stored feature information
   - Preserved transformation parameters

## Next Steps:
1. Proceed to model development (03_churn_modeling.ipynb)
2. Evaluate feature importance
3. Develop prediction models
4. Validate model performance

In [31]:
# Export preprocessed data and related artifacts
import os

# Create processed data directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

# 1. Export preprocessed dataset
df_scaled.to_csv('../data/processed/preprocessed_churn_data.csv', index=False)
print("\nPreprocessed dataset saved to: ../data/processed/preprocessed_churn_data.csv")

# 2. Export categorical encoders for deployment
categorical_features = {
    'value_segment': df['value_segment'].unique().tolist(),
    'recency_segment': df['recency_segment'].unique().tolist(),
    'risk_segment': df['risk_segment'].unique().tolist()
}

# Save encoders
with open('../models/encoders.json', 'w') as f:
    json.dump(categorical_features, f, indent=4)
print("Categorical encoders saved to: ../models/encoders.json")

# 3. Export feature metadata
feature_metadata = {
    'numerical_features': [col for col in df_scaled.select_dtypes(include=['float64', 'int64']).columns 
                         if col not in ['visitorid', 'target_class']],
    'categorical_features': list(categorical_features.keys()),
    'target_variable': 'target_class',
    'id_column': 'visitorid'
}

# Save feature metadata
with open('../models/model_metadata.json', 'w') as f:
    json.dump(feature_metadata, f, indent=4)
print("Feature metadata saved to: ../models/model_metadata.json")

# Display summary of exported files
print("\nEXPORTED FILES SUMMARY:")
print("-" * 40)
print(f"1. Preprocessed Data Shape: {df_scaled.shape}")
print("\n2. Categorical Features:")
for feature, values in categorical_features.items():
    print(f"   - {feature}: {len(values)} unique values")
print("\n3. Feature Counts:")
print(f"   - Numerical Features: {len(feature_metadata['numerical_features'])}")
print(f"   - Categorical Features: {len(feature_metadata['categorical_features'])}")


Preprocessed dataset saved to: ../data/processed/preprocessed_churn_data.csv
Categorical encoders saved to: ../models/encoders.json
Feature metadata saved to: ../models/model_metadata.json

EXPORTED FILES SUMMARY:
----------------------------------------
1. Preprocessed Data Shape: (49358, 69)

2. Categorical Features:
   - value_segment: 4 unique values
   - recency_segment: 4 unique values
   - risk_segment: 5 unique values

3. Feature Counts:
   - Numerical Features: 61
   - Categorical Features: 3
